## Encodage des récompenses

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Récompenses pour le codage

- Nous avons maintenant discuté de l'importance de coder les observations.
- Nous pouvons aussi avoir un certain choix sur l'espace d'action, bien qu'ici (et souvent) il soit relativement clair/fixe.
- Mais qu'en est-il des récompenses ? 

#### Mise en place du courant

- Actuellement, nous recevons une récompense de +1 pour avoir atteint l'objectif 
- C'est une partie de ce qui rend RL si difficile (et impressionnant) :
  - Nous voulons apprendre des actions même si nous ne savons pas tout de suite si l'action était "bonne" 
  - Compare cela à l'apprentissage supervisé, où chaque prédiction que nous faisons sur les données de formation peut immédiatement être comparée à la valeur cible connue.


In [2]:
# TODO: perhaps this next slide can be moved to Module 1, since it's very general?

#### Les agents ne peuvent pas être simplement gourmands

- Les agents peuvent-ils simplement apprendre à rechercher la meilleure récompense immédiate ?
- Non. Par exemple, dans un système de recommandation de vidéos, montrer à l'utilisateur une autre vidéo de chat drôle pourrait le faire cliquer (récompense immédiate élevée) mais entraîner une perte d'intérêt à long terme pour le service (faible récompense à long terme).
- Notre lac gelé est un autre exemple de ce problème : parfois, il n'y a pas du tout de récompense immédiate dont on peut tirer un enseignement.

In [3]:
# TODO: perhaps this next section on "Learned action probabilities" could be moved much earlier, even as early as Module 1

#### Probabilités d'action apprises

- RLlib nous permet de regarder à l'intérieur du modèle la probabilité de chaque action compte tenu d'une observation (c'est-à-dire la politique apprise).
- Chargeons le modèle formé avec nos observations codées :

In [4]:
from envs_03 import RandomLakeObs
from ray.rllib.algorithms.ppo import PPOConfig

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")
)
ppo_RandomLakeObs = ppo_config.build(env=RandomLakeObs)

In [5]:
# # HIDDEN

# for i in range(16):
#     ppo_RandomLakeObs.train()
    
# print(ppo_RandomLakeObs.evaluate()["evaluation"]["episode_reward_mean"])

# ppo_RandomLakeObs.save("models/RandomLakeObs-Ray2")

In [6]:
ppo_RandomLakeObs.restore("models/RandomLakeObs-Ray2/checkpoint_000016")

#### Probabilités d'action apprises

Nous utiliserons la fonction `query_Policy` du module 2 :

In [7]:
from utils_03 import query_policy
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,0,0,0])

array([0.00902206, 0.5078786 , 0.47434822, 0.00875122], dtype=float32)

- Rappelle-toi l'ordonnancement (gauche, bas, droite, haut).
- Lorsque l'observation est `[0 0 0 0]` (pas de trous ou de bords en vue), l'agent préfère aller vers le bas et la droite.

Et s'il y a un trou en dessous de toi ? Nous pouvons introduire une observation différente dans la politique :

In [8]:
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,1,0,0])

array([0.01965651, 0.00299437, 0.9645823 , 0.01276694], dtype=float32)

- Maintenant, il est très peu probable que l'agent descende et très probable qu'il aille à droite !
- Encore une fois, tout cela a été appris par essais et erreurs, avec une récompense obtenue uniquement lorsque l'objectif a été atteint.

#### Récompenses Random Lake

- Dans l'exemple de Random Lake, ne peut-on pas faciliter la vie de l'agent en lui donnant des récompenses immédiates ?

Voici le code de récompense actuel :

In [9]:
def reward(self):
    return int(self.player == self.goal)

- L'agent doit apprendre, par essais et erreurs au cours de _vastes épisodes_, que se déplacer vers le bas et la droite est généralement une bonne chose 

#### Redéfinir les récompenses

- Essayons plutôt de donner une récompense _à chaque étape, qui est plus élevée à mesure que l'agent se rapproche de l'objectif_ 

In [10]:
from envs_03 import RandomLakeObs

class RandomLakeObsRew(RandomLakeObs):
    def reward(self):
        return 6-(abs(self.player[0]-self.goal[0]) + abs(self.player[1]-self.goal[1]))

- La méthode ci-dessus utilise la [Distance de Manhattan] (https://en.wikipedia.org/wiki/Taxicab_geometry) entre le joueur et l'objectif comme récompense 
- Lorsque l'agent atteint l'objectif, la récompense maximale de 6 est obtenue.
- Lorsque l'agent est le plus éloigné du but, la récompense minimale de 0 est donnée.

#### Redéfinir les récompenses

In [11]:
env = RandomLakeObsRew()
env.reset()
env.render()
env.reward()

🧑🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


0

⬆️ la récompense est de 0

⬇️ la récompense est de 1 car nous nous sommes rapprochés de l'objectif

In [12]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧑🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


1

#### Redéfinir les récompenses

In [13]:
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧑
🧊🧊🧊⛳️


5

Maintenant, la récompense est de 5. La prochaine fois, elle sera de 6.

In [14]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊🧑


6

#### Comparer les récompenses

- Nous avons donc deux fonctions de récompense possibles. Laquelle fonctionne le mieux ? 
- Rappelle-toi que la dernière fois, après un entraînement de 8 itérations, nous avons réussi à atteindre l'objectif environ 70 % du temps :

In [15]:
ppo_RandomLakeObs.evaluate()['evaluation']['episode_reward_mean']

0.8166666666666667

#### Comparer les récompenses

Entraîne-toi avec la nouvelle fonction de récompense !

In [16]:
ppo_RandomLakeObsRew = ppo_config.build(env=RandomLakeObsRew)

In [17]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [18]:
ppo_RandomLakeObsRew.evaluate()['evaluation']['episode_reward_mean']

101.90566037735849

Attends une minute, qu'est-ce qui se passe ici ?

#### Comparer les récompenses ?

- Nous avons essayé d'améliorer notre système RL en façonnant la fonction de récompense.
- Cela a (vraisemblablement) affecté la formation, mais aussi notre évaluation.
- Dans l'apprentissage supervisé, cela revient à changer la métrique de notation de l'erreur quadratique à l'erreur absolue.
- Si l'ancien système a obtenu une erreur quadratique moyenne de 20 000 et le nouveau système une erreur absolue moyenne de 40, lequel est le meilleur ?
- Nous comparons des pommes et des oranges ici !
- Nous voulons comparer les deux modèles sur la même métrique, par exemple la métrique originale 
- Ici, nous voulons voir à quelle fréquence l'agent atteint l'objectif.

#### Comparer les récompenses ?

- Le code ici est un peu plus avancé.
- Il est inclus par souci d'exhaustivité, mais nous n'entrerons pas dans les détails.

In [19]:
from ray.rllib.agents.callbacks import DefaultCallbacks

class MyCallbacks(DefaultCallbacks):
    def on_episode_end(self, *, worker, base_env, policies, episode, env_index, **kwargs):
        info = episode.last_info_for()
        episode.custom_metrics["goal_reached"] = info["player"] == info["goal"]

In [20]:
ppo_config_callback = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")\
    .callbacks(callbacks_class=MyCallbacks)\
    .evaluation(evaluation_config={"callbacks" : MyCallbacks})
)

ppo_RandomLakeObsRew = ppo_config_callback.build(env=RandomLakeObsRew)

Le formateur ci-dessus utilise notre nouveau système de récompense mais signale/mesure également le taux d'atteinte de l'objectif.

#### Comparer les récompenses ?

Essayons-le !

In [21]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [22]:
# HIDDEN
ppo_RandomLakeObsRew.evaluate()["evaluation"]["episode_reward_mean"]

101.90566037735849

In [23]:
ppo_RandomLakeObsRew.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.04081632653061224

- Hmm, ces résultats sont terribles !
- Nous avions l'habitude d'obtenir un taux de victoire de plus de 70%, et maintenant nous sommes proches de zéro.
- Que s'est-il passé ? 🤔

#### Qu'est-ce que l'agent optimise vraiment ?

- L'agent optimise vraiment la _récompense totale_.
- _Total_ : il valorise toutes les récompenses qu'il collecte, pas seulement la récompense finale.
- _Récompensé_ : il valorise davantage les premières récompenses que les dernières.
- Notre agent réussit à maximiser la récompense totale actualisée, mais cela ne correspond pas à l'atteinte de l'objectif.
- Mais pourquoi ? L'objectif donne une récompense plus élevée.

#### Exploration vs. exploitation

- Un concept fondamental en RL est _exploration vs. exploitation_
- Lorsque l'agent apprend la politique, il peut choisir de soit :

1. Faire des choses qu'il sait être assez bonnes ("exploiter")
2. Essayer quelque chose de totalement nouveau et fou, juste au cas où ("explorer")

In [24]:
# TODO 
# diagram for this?

#### Exploration vs. exploitation

- Avec l'ancienne structure de récompense, l'agent reçoit une récompense de 0 à moins qu'il n'atteigne l'objectif.
  - Il continue donc à essayer de trouver quelque chose de mieux.
- Avec la nouvelle structure de récompense, l'agent reçoit beaucoup de récompenses simplement parce qu'il se promène.
  - Il n'est pas très motivé pour explorer l'environnement.
- En fait, comme il maximise la récompense **totale** actualisée, trouver le but est une mauvaise chose !
  - Cela entraîne la fin de l'épisode, ce qui limite la récompense totale de l'agent.
  - L'agent apprend en fait à _éviter_ le but, surtout au début de l'épisode.

#### Concevoir une meilleure structure de récompense

- Essayons plutôt de pénaliser l'agent lorsqu'il entre dans un trou ou sort du bord.
- Il sera plus facile d'implémenter cela directement dans `step` :

In [25]:
class RandomLakeObsRew2(RandomLakeObs):
    def step(self, action):
        # (not shown) existing code gets new_loc, where the player is trying to go
        
        reward = 0
        
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        else:
            reward -= 0.1 # small penalty
            
        if self.holes[self.player]:
            reward -= 0.1 # small penalty
            
        if self.player == self.goal:
            reward += 1
        
        # Return observation/reward/done
        return self.observation(), reward, self.done(), {"player" : self.player, "goal" : self.goal}

In [26]:
# HIDDEN
from envs_03 import RandomLakeObsRew2

#### Teste-le, encore une fois

In [27]:
# HIDDEN
# redefine ppo_RandomLakeObs to include the new callbacks
# so that you can measure the custom metric instead of the reward
# they will give the same value but this is better for consistency
ppo_RandomLakeObs = ppo_config_callback.build(env=RandomLakeObs)

for i in range(8):
    ppo_RandomLakeObs.train()

In [30]:
ppo_RandomLakeObsRew2 = ppo_config_callback.build(env=RandomLakeObsRew2)

In [31]:
for i in range(8):
    ppo_RandomLakeObsRew2.train()

In [32]:
ppo_RandomLakeObs.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.6853448275862069

In [33]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.734982332155477

Il semble que, cette fois, les deux méthodes ont des performances beaucoup plus similaires.

#### Durée de l'épisode

- En plus du taux de réussite, nous pouvons calculer d'autres statistiques sur le comportement de l'agent.
- Une mesure intéressante est la longueur des épisodes.
- RLlib l'enregistre par défaut, nous pouvons donc y accéder facilement :

In [34]:
ppo_RandomLakeObs.evaluate()["evaluation"]["episode_len_mean"]

8.585470085470085

In [35]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["episode_len_mean"]

7.20216606498195

Bien que les deux agents aient le même taux de réussite, le nouveau tend vers des épisodes plus courts.

Notes 

- C'est très intéressant car l'agent ne peut pas "voir" la différence entre les trous et les bords.
- Nous pourrions approfondir cette question en ajoutant d'autres mesures personnalisées, par exemple le nombre de bosses dans l'arête.

In [ ]:
# TODO
#### disadvantages - loss of generality

#- now only works if goal is at bottom-right
#give a few real-world examples here -> important

## Analogie de l'apprentissage supervisé : façonnage de la récompense
<!-- multiple choice -->

Plus tôt, nous avons fait une analogie entre l'encodage des observations en RL et le prétraitement des caractéristiques en apprentissage supervisé. Quel aspect de l'apprentissage supervisé est la meilleure analogie avec la mise en forme des récompenses en RL ?

- [ ] Ingénierie des caractéristiques 
- [ ] Sélection de modèles [ ] Pas tout à fait. Mais, comme nous le verrons, la sélection de modèles a aussi sa place dans l'apprentissage supervisé !
- [ ] Réglage des hyperparamètres [ ] Pas tout à fait. Mais, comme nous le verrons, l'ajustement des hyperparamètres a également sa place dans RL !
- [Sélection d'une fonction de perte | Changer la fonction de perte change le "meilleur" modèle, tout comme changer les récompenses change la "meilleure" politique.

## Récompenser chaque étape : petites récompenses négatives
<!-- multiple choice -->

Dans des environnements RL comme Random Lake où l'agent doit atteindre un objectif spécifique, imagine que nous attribuions une minuscule récompense négative pour _chaque_ étape effectuée par l'agent. Comment cela affecterait-il généralement/typiquement le temps que l'agent passe jusqu'à ce qu'il atteigne l'objectif ?

- [ ] L'agent essaiera d'atteindre l'objectif en faisant le moins d'étapes possible.
- [ ] L'agent essaiera d'atteindre l'objectif en autant d'étapes que possible. | [ ] Si nous pénalisons chaque étape, le fait de faire plus d'étapes entraînera une récompense moindre.
- [ ] Aucun changement. | Si nous pénalisons chaque étape, le fait de faire plus d'étapes entraînera une récompense moindre.

## Exploration vs. exploitation
<!-- multiple choice -->

Laquelle des affirmations suivantes est correcte concernant le compromis exploration-exploitation dans RL ?

- [ ] Si une personne n'explore que, elle ne trouvera jamais une bonne politique. | Il trouvera de bonnes politiques en fait, mais EXTRÊMEMENT lentement.
- [ ] Si un agent ne fait qu'exploiter, il ne trouvera jamais de bonne politique. | Il peut simplement continuer à essayer la même chose encore et encore.
- [ ] Les agents trouvent toujours de bonnes politiques même sans exploration/exploitation.

## Conséquences involontaires
<!-- coding exercise -->

Dans cet exercice, tu vas essayer une mauvaise idée : attribuer une grande récompense négative chaque fois que l'agent fait un pas. Nous utiliserons -1 par étape. L'agent reçoit quand même une récompense de +1 pour avoir atteint l'objectif. Mets en place cette récompense, entraîne l'agent et regarde la longueur moyenne des épisodes imprimée par le code. Compare-le à la durée moyenne des épisodes d'un agent qui agit simplement de manière aléatoire. Ensuite, réponds à la question à choix multiple sur le comportement de l'agent. À ton avis, que se passe-t-il ici ? 

(Pour info : comme nous l'avons vu précédemment, ce type de modification d'un environnement peut aussi être réalisé avec des wrappers de gymnastique)

In [ ]:
# EXERCISE
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return ____
    
ppo = lake_default_config.build(env=____)

for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"][____])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"][____])

In [2]:
# SOLUTION
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return old_reward - 1

ppo = lake_default_config.build(env=RandomLakeBadIdea)


for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"]["episode_len_mean"])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"]["episode_len_mean"])

0
1
2
3
4
5
6
7
Average episode length for trained agent: 4.4
Average episode length for random agent: 12.1


#### Comportement de l'agent

Lorsqu'il est entraîné dans un environnement avec une grande récompense négative à chaque étape, que crois-tu que cet agent fait, qui est indésirable ?

- [L'agent reste immobile car on le décourage de bouger. | Essaie encore !
- [ ] L'agent n'est pas intéressé par l'atteinte de l'objectif car la récompense est relativement faible. | Essaie encore !
- [ ] L'agent apprend à sauter dans le lac aussi vite qu'il le peut, pour éviter la récompense négative du mouvement. | Yikes ! 🥶
- [ ] L'agent atteint l'objectif tout de suite. | Ce serait pourtant souhaitable !